## Harbor Installation

Dieses Notebook enthält die notwendigen Befehle zur Konfiguration von Harbor mittels Docker Compose.

Folgendes wurde bereites Installiert bzw. downgeloadet
* Harbor nach ~/harbor
* docker-compose
* cert-manager 
    * SelfSigned ClusterIssuer (stellt die CA aus)
    * Certificate für CA
    * CA Issuer (nutzt die erzeugte CA)
    * Das eigentliches Zertifikat


Auslesen der Zertifikate aus Kubernetes

In [ ]:
%%bash
kubectl get secret root-selfsigned-cert -o jsonpath="{.data.tls\.crt}" | base64 --decode > ~/harbor/tls.crt
kubectl get secret root-selfsigned-cert -o jsonpath="{.data.tls\.key}" | base64 --decode > ~/harbor/tls.key

Zertifikatsinhalt prüfen bzw. Ausgeben

In [ ]:
%%bash
openssl x509 -in ~/harbor/tls.crt -text -noout

### Harbor Konfiguration anpassen

In [ ]:
%%bash
cd ~/harbor
cp harbor.yml.tmpl harbor.yml
sed -i -e "s/reg.mydomain.com/$(cat ~/work/server-ip)/g" harbor.yml
sed -i -e 's/port: 80/port: 9090/g' harbor.yml
sed -i -e 's/port: 443/port: 9443/g' harbor.yml
sed -i -e 's;certificate: /your/certificate/path;certificate: /home/ubuntu/harbor/tls.crt;g' harbor.yml
sed -i -e 's;private_key: /your/private/key/path;private_key: /home/ubuntu/harbor/tls.key;g' harbor.yml

### Harbor starten

In [ ]:
%%bash
cd ~/harbor
sudo ./install.sh --with-trivy

Harbor ist dann via dem untenstehenden URL erreichbar.
Benutzer: `admin`, Passwort: `Harbor12345`.

In [ ]:
%%bash
echo "https://$(cat ~/work/server-ip):9443"

### Docker: Selbst signiertes Zertifikat vertrauenswürdig machen

In [ ]:
%%bash
kubectl get secret root-selfsigned-cert -o jsonpath="{.data.ca\.crt}" | base64 --decode > ~/harbor/ca.crt

In [ ]:
%%bash
sudo mkdir -p /etc/docker/certs.d/$(cat ~/work/server-ip):9443
sudo cp ~/harbor/ca.crt /etc/docker/certs.d/$(cat ~/work/server-ip):9443/ca.crt

Jetzt sollte `docker login` ohne Fehlermeldung funktionieren.

In [ ]:
%%bash
docker login $(cat ~/work/server-ip):9443 -u admin -p Harbor12345

### Registry Harbor verwenden

Dazu builden wir zuerst unsere Auto Shop Microservices

In [ ]:
%%bash
rm -rf shop
git clone --branch v2.1.0 https://gitlab.com/ch-mc-b/autoshop-ms/app/shop.git
cd shop
sed -i -e "s;registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop;$(cat ~/work/server-ip):9443/library;g" docker-compose.yml
docker-compose build


Und pushen die Microservices in die Registry

In [ ]:
%%bash
cd shop
docker image ls | grep $(cat ~/work/server-ip)
docker-compose push

Weiter geht es mit dem Deployment auf Kubernetes

In [ ]:
%%bash
# Clean
sudo rm -rf kubernetes-templates/
# Kubernetes Deklarationen
git clone https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates.git
cp -r kubernetes-templates/2.1.0-deployment/ shop/k8s/
cp kubernetes-templates/*-service.yaml kubernetes-templates/*-ingress.yaml shop/k8s/
# Registry von Gitlab auf Harbor Registry umstellen
for yaml in shop/k8s/*-deployment.yaml
do
    sed -i -e "s;registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop;$(cat ~/work/server-ip):9443/library;g" ${yaml}
done
sudo rm -rf kubernetes-templates shop/.git shop/docker-compose.yml shop/m1
cat shop/k8s/order-deployment.yaml

In [ ]:
%%bash
kubectl create namespace shop-harbor
kubectl apply --namespace shop-harbor -f shop/k8s

***

### Aufträge
- Repliziert Eure Container Registries bzw. die Images nach Harbor.
- Erstellt ein eigenes Container Image und legt es in die Harbor Registry ab.
- Legt ein helm Chart in der Harbor Registry ab

**Links:**
[Harbor Installer](https://goharbor.io/docs/2.12.0/install-config/run-installer-script/)

- - - 

### Aufräumen

In [ ]:
%%bash
kubectl delete --namespace shop-harbor -f shop/k8s
kubectl delete namespace shop-harbor 
sudo rm -rf shop kubernetes-templates/